# v2

In [4]:
import torch
import torch.nn as nn
import torch_pruning as tp
import matplotlib.pyplot as plt
from torch import optim
import os
import numpy as np
import copy
import json
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import MinMaxScaler
from typing import List, Tuple
import warnings
warnings.filterwarnings('ignore')

# Configuration
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
MODEL_BASE_NAME = "mlp_nasa"
print(f"Using device: {DEVICE}")

# SIMPLIFIED preprocessing based on proven NASA C-MAPSS research
column_names = ['unit_number', 'time_in_cycles'] + [f'op_setting_{i}' for i in range(1, 4)] + [f'sensor_{i}' for i in range(1, 24)]

def load_dataframe(file_path: str) -> pd.DataFrame | None:
    """Load NASA data file."""
    try:
        df = pd.read_csv(file_path, sep=' ', header=None, names=column_names)
        df.dropna(axis=1, how='all', inplace=True)
        return df
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def proven_feature_selection(df: pd.DataFrame) -> list:
    """Use PROVEN feature selection from NASA literature - SIMPLE approach."""
    if df is None:
        return []

    # Based on multiple research papers - these are the PROVEN effective sensors for FD001
    # Remove sensors with minimal variation (proven approach)
    sensors_to_remove = [
        'sensor_1',   # Lever position - constant for FD001
        'sensor_5',   # Static pressure - minimal variation
        'sensor_6',   # Physical fan speed - redundant with sensor_8
        'sensor_10',  # Static pressure - minimal variation
        'sensor_16',  # Static pressure - minimal variation
        'sensor_18',  # Bleed enthalpy - minimal variation
        'sensor_19'   # Demanded fan speed - minimal variation
    ]

    # Remove operational settings - constant for FD001
    ops_to_remove = ['op_setting_1', 'op_setting_2', 'op_setting_3']

    cols_to_remove = sensors_to_remove + ops_to_remove

    print(f"Removing {len(cols_to_remove)} non-informative features")
    print(f"Keeping sensors: [2,3,4,7,8,9,11,12,13,14,15,17,20,21]")
    return cols_to_remove

def add_rul_simple(df: pd.DataFrame) -> pd.DataFrame | None:
    """Simple but PROVEN RUL calculation."""
    if df is None:
        return None

    max_cycles = df.groupby('unit_number')['time_in_cycles'].max().reset_index()
    max_cycles.columns = ['unit_number', 'max_cycle']
    df = df.merge(max_cycles, on='unit_number', how='left')
    df['RUL'] = df['max_cycle'] - df['time_in_cycles']
    df.drop(columns=['max_cycle'], inplace=True)

    # PROVEN: Piece-wise linear RUL - engines don't degrade early in life
    # This is the STANDARD approach in NASA research
    df['RUL'] = df['RUL'].apply(lambda x: min(x, 125))

    return df

def normalize_data_simple(df: pd.DataFrame, columns_to_normalize: List[str],
                         scaler: MinMaxScaler = None) -> Tuple[pd.DataFrame, MinMaxScaler]:
    """Simple normalization that works."""
    if df is None:
        return None, None

    data_to_scale = df[columns_to_normalize]

    if scaler is None:
        scaler = MinMaxScaler(feature_range=(0, 1))
        df[columns_to_normalize] = scaler.fit_transform(data_to_scale)
    else:
        df[columns_to_normalize] = scaler.transform(data_to_scale)

    return df, scaler

def prepare_cmapss_data_simple(data_dir: str, train_file: str, test_file: str, test_rul_file: str) -> Tuple[
    pd.DataFrame, pd.DataFrame, pd.DataFrame, MinMaxScaler, List[str]]:
    """SIMPLE but PROVEN data preparation."""
    print("--- Simple Proven Data Preparation ---")

    # Load data
    train_df = load_dataframe(os.path.join(data_dir, train_file))
    test_df = load_dataframe(os.path.join(data_dir, test_file))
    test_rul_df = pd.read_csv(os.path.join(data_dir, test_rul_file), header=None, names=['RUL'])

    # Add RUL
    train_df = add_rul_simple(train_df)

    # Simple feature selection - PROVEN approach
    cols_to_remove = proven_feature_selection(train_df)
    feature_cols = [col for col in train_df.columns if
                   col not in ['unit_number', 'time_in_cycles', 'RUL'] + cols_to_remove]

    # Remove unwanted columns
    train_df.drop(columns=cols_to_remove, inplace=True, errors='ignore')
    test_df.drop(columns=cols_to_remove, inplace=True, errors='ignore')

    print(f"Using {len(feature_cols)} proven effective sensors")

    # Simple normalization
    train_df_norm, scaler = normalize_data_simple(train_df.copy(), feature_cols)
    test_df_norm, _ = normalize_data_simple(test_df.copy(), feature_cols, scaler=scaler)

    return train_df_norm, test_df_norm, test_rul_df, scaler, feature_cols

# SIMPLE Dataset - no over-engineering
class NASADatasetSimple(Dataset):
    def __init__(self, df: pd.DataFrame, feature_cols: List[str], window_size: int = 30,
                 stride: int = 1, is_test: bool = False, test_rul_df: pd.DataFrame = None):
        self.df = df
        self.feature_cols = feature_cols
        self.window_size = window_size
        self.stride = stride
        self.is_test = is_test
        self.test_rul_df = test_rul_df
        self.samples = []
        self.targets = []

        self._prepare_samples()

    def _prepare_samples(self):
        """SIMPLE windowing strategy - PROVEN approach."""
        units = self.df['unit_number'].unique()

        for unit in units:
            unit_df = self.df[self.df['unit_number'] == unit].sort_values('time_in_cycles')

            if self.is_test:
                # Test: only last window
                if len(unit_df) >= self.window_size:
                    window_data = unit_df[self.feature_cols].iloc[-self.window_size:].values
                    self.samples.append(window_data)
                    if self.test_rul_df is not None:
                        rul = min(self.test_rul_df.iloc[unit - 1]['RUL'], 125)
                        self.targets.append(rul)
                else:
                    # Pad if needed
                    window_data = unit_df[self.feature_cols].values
                    padded = np.zeros((self.window_size, len(self.feature_cols)))
                    if len(window_data) > 0:
                        padded[-len(window_data):] = window_data
                    self.samples.append(padded)
                    if self.test_rul_df is not None:
                        rul = min(self.test_rul_df.iloc[unit - 1]['RUL'], 125)
                        self.targets.append(rul)
            else:
                # Training: simple sliding window
                for i in range(0, len(unit_df) - self.window_size + 1, self.stride):
                    window_data = unit_df[self.feature_cols].iloc[i:i + self.window_size].values
                    rul = unit_df['RUL'].iloc[i + self.window_size - 1]
                    self.samples.append(window_data)
                    self.targets.append(rul)

        self.samples = np.array(self.samples, dtype=np.float32)
        self.targets = np.array(self.targets, dtype=np.float32)

        print(f"Created {len(self.samples)} samples with simple windowing")

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        sample = self.samples[idx].flatten()
        target = self.targets[idx]
        return torch.FloatTensor(sample), torch.FloatTensor([target])

# SIMPLE MLP - proven architecture
class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes=[80, 40], dropout_rate=0.2):
        super(SimpleMLP, self).__init__()

        layers = []
        prev_size = input_size

        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(prev_size, hidden_size),
                nn.ReLU(),
                nn.Dropout(dropout_rate)
            ])
            prev_size = hidden_size

        # Output layer - NO activation for regression
        layers.append(nn.Linear(prev_size, 1))

        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def get_data_loaders_simple(data_dir='./data/NASA', batch_size=32, window_size=30, val_split=0.2, seed=42):
    """Simple data loading - proven parameters."""
    print(f"Loading NASA C-MAPSS dataset from: {data_dir}")

    train_df, test_df, test_rul_df, scaler, feature_cols = prepare_cmapss_data_simple(
        data_dir, 'train_FD001.txt', 'test_FD001.txt', 'RUL_FD001.txt'
    )

    # Create datasets with simple approach
    full_train_dataset = NASADatasetSimple(train_df, feature_cols, window_size=window_size, stride=1)

    # Train/val split
    val_size = int(len(full_train_dataset) * val_split)
    train_size = len(full_train_dataset) - val_size
    generator = torch.Generator().manual_seed(seed)
    train_dataset, val_dataset = torch.utils.data.random_split(
        full_train_dataset, [train_size, val_size], generator=generator
    )

    test_dataset = NASADatasetSimple(test_df, feature_cols, window_size=window_size,
                                    is_test=True, test_rul_df=test_rul_df)

    # Simple data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    input_size = window_size * len(feature_cols)
    print(f"Input size: {input_size} (window: {window_size} × features: {len(feature_cols)})")
    print(f"Train: {len(train_dataset)}, Val: {len(val_dataset)}, Test: {len(test_dataset)}")

    return train_loader, val_loader, test_loader, input_size

def get_simple_mlp_model(input_size, hidden_sizes=[80, 40], dropout_rate=0.2):
    """Simple but effective MLP."""
    model = SimpleMLP(input_size, hidden_sizes, dropout_rate)
    print(f"✅ Created Simple MLP: {input_size} -> {' -> '.join(map(str, hidden_sizes))} -> 1")
    return model

def get_ignored_layers(model):
    """Get final layer to ignore during pruning."""
    ignored_layers = []
    for module in reversed(list(model.model)):
        if isinstance(module, nn.Linear):
            ignored_layers.append(module)
            break
    return ignored_layers

def calculate_macs_params(model, example_input):
    """Calculate efficiency metrics."""
    model.eval()
    target_device = example_input.device
    model_on_device = model.to(target_device)

    with torch.no_grad():
        macs, params = tp.utils.count_ops_and_params(model_on_device, example_input)

    return macs, params

def save_model(model, save_path, example_input_cpu=None):
    """Save model."""
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    torch.save(model.state_dict(), save_path)
    print(f"✅ Model saved to {save_path}")

def evaluate_model(model, data_loader, example_input, criterion, device):
    """Simple evaluation."""
    model.eval()
    model.to(device)

    # Calculate efficiency metrics
    macs, params = calculate_macs_params(model, example_input.to(device))
    model_size_mb = params * 4 / (1024 * 1024)

    # Calculate performance metrics
    total_loss = 0.0
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)

            total_loss += loss.item() * data.size(0)
            all_predictions.extend(output.cpu().numpy())
            all_targets.extend(target.cpu().numpy())

    all_predictions = np.array(all_predictions).flatten()
    all_targets = np.array(all_targets).flatten()

    mse = np.mean((all_predictions - all_targets) ** 2)
    mae = np.mean(np.abs(all_predictions - all_targets))

    return {
        'mse': mse,
        'mae': mae,
        'loss': total_loss / len(data_loader.dataset),
        'macs': macs,
        'params': params,
        'size_mb': model_size_mb
    }

def prune_model(model, strategy_config, sparsity_ratio, example_input, ignored_layers=None):
    """Simple pruning."""
    if sparsity_ratio == 0.0:
        return model

    model.eval()
    pruned_model = copy.deepcopy(model)
    pruned_model.to(example_input.device)

    initial_macs, _ = calculate_macs_params(pruned_model, example_input)
    print(f"Initial MACs: {initial_macs / 1e6:.2f}M")

    ignored_layers = ignored_layers or []

    # Simple pruner
    pruner = strategy_config['pruner'](
        pruned_model,
        example_input,
        importance=strategy_config['importance'],
        iterative_steps=3,  # Fewer steps
        ch_sparsity=sparsity_ratio,
        root_module_types=[nn.Linear],
        ignored_layers=ignored_layers
    )

    print(f"Applying {strategy_config['importance'].__class__.__name__} pruning at {sparsity_ratio:.1%}")
    pruner.step()

    final_macs, _ = calculate_macs_params(pruned_model, example_input)
    reduction = (initial_macs - final_macs) / initial_macs * 100 if initial_macs > 0 else 0
    print(f"Final MACs: {final_macs / 1e6:.2f}M ({reduction:.1f}% reduction)")

    return pruned_model

def train_model_simple(model, train_loader, criterion, optimizer, device, num_epochs,
                      val_loader=None, patience=10, log_prefix=""):
    """SIMPLE but effective training."""
    model.to(device)

    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model_state = None

    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0

        for data, target in train_loader:
            data, target = data.to(device), target.to(device)

            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        avg_train_loss = train_loss / len(train_loader)

        # Validation
        if val_loader:
            model.eval()
            val_loss = 0.0

            with torch.no_grad():
                for data, target in val_loader:
                    data, target = data.to(device), target.to(device)
                    output = model(data)
                    loss = criterion(output, target)
                    val_loss += loss.item()

            avg_val_loss = val_loss / len(val_loader)

            # Early stopping
            if avg_val_loss < best_val_loss:
                best_val_loss = avg_val_loss
                epochs_no_improve = 0
                best_model_state = copy.deepcopy(model.state_dict())
            else:
                epochs_no_improve += 1

            if epoch % 50 == 0:
                print(f"Epoch {epoch+1}: Train: {avg_train_loss:.4f}, Val: {avg_val_loss:.4f}")

            if epochs_no_improve >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    if best_model_state is not None:
        model.load_state_dict(best_model_state)
        print("Loaded best model state")

    return model, {}

def save_results_to_files(all_results, output_dir):
    """Save results."""
    os.makedirs(output_dir, exist_ok=True)

    # JSON
    with open(os.path.join(output_dir, 'complete_results.json'), 'w') as f:
        json.dump(all_results, f, indent=2, default=str)

    # CSV
    summary_data = []
    for strategy, strategy_results in all_results.items():
        for sparsity, metrics in strategy_results.items():
            row = {
                'strategy': strategy,
                'sparsity_ratio': sparsity,
                'mse': metrics['mse'],
                'mae': metrics['mae'],
                'loss': metrics['loss'],
                'macs_millions': metrics['macs'] / 1e6,
                'params_millions': metrics['params'] / 1e6,
                'size_mb': metrics['size_mb']
            }
            summary_data.append(row)

    summary_df = pd.DataFrame(summary_data)
    summary_df.to_csv(os.path.join(output_dir, 'summary_results.csv'), index=False)

    return summary_df

def create_results_plots(summary_df, output_dir):
    """Create plots."""
    os.makedirs(output_dir, exist_ok=True)

    # MSE vs Sparsity
    plt.figure(figsize=(10, 6))
    for strategy in summary_df['strategy'].unique():
        data = summary_df[summary_df['strategy'] == strategy].sort_values('sparsity_ratio')
        plt.plot(data['sparsity_ratio'] * 100, data['mse'], 'o-', label=strategy, linewidth=2, markersize=8)

    plt.xlabel('Sparsity (%)')
    plt.ylabel('MSE')
    plt.title('NASA MLP: MSE vs Sparsity (Simple Approach)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'mse_vs_sparsity.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # Efficiency frontier
    plt.figure(figsize=(10, 6))
    for strategy in summary_df['strategy'].unique():
        data = summary_df[summary_df['strategy'] == strategy].sort_values('sparsity_ratio')
        plt.scatter(data['macs_millions'], data['mse'], label=strategy, s=100, alpha=0.8)
        plt.plot(data['macs_millions'], data['mse'], '--', alpha=0.6)

    plt.xlabel('MACs (Millions)')
    plt.ylabel('MSE')
    plt.title('NASA MLP: Efficiency Frontier (Simple Approach)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'efficiency_frontier.png'), dpi=300, bbox_inches='tight')
    plt.close()

def print_results_table(summary_df):
    """Print results table."""
    print("\n" + "=" * 80)
    print("SIMPLE APPROACH RESULTS")
    print("=" * 80)

    # Baseline
    baseline = summary_df[summary_df['sparsity_ratio'] == 0.0].iloc[0]
    print(f"\nBaseline Performance:")
    print(f"  MSE: {baseline['mse']:.2f}")
    print(f"  MAE: {baseline['mae']:.2f}")
    print(f"  MACs: {baseline['macs_millions']:.2f}M")
    print(f"  Parameters: {baseline['params_millions']:.2f}M")

    # Complete table
    print(f"\nComplete Results:")
    print("-" * 80)
    print(f"{'Strategy':<12} {'Sparsity':<8} {'MSE':<8} {'MAE':<8} {'MACs(M)':<8} {'Params(M)':<9} {'Size(MB)':<8}")
    print("-" * 80)

    for _, row in summary_df.sort_values(['strategy', 'sparsity_ratio']).iterrows():
        print(f"{row['strategy']:<12} {row['sparsity_ratio']*100:>6.0f}% "
              f"{row['mse']:>7.2f} {row['mae']:>7.2f} {row['macs_millions']:>7.2f} "
              f"{row['params_millions']:>8.2f} {row['size_mb']:>7.2f}")

def main():
    """SIMPLE approach - back to basics with research-proven optimizations."""
    print("Starting SIMPLE NASA MLP Approach - Back to Proven Basics")
    print("=" * 60)

    # SIMPLE configuration - proven parameters from literature
    config = {
        'strategies': {
            'MagnitudeL2': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.MagnitudeImportance(p=2)
            },
            'Random': {
                'pruner': tp.pruner.MagnitudePruner,
                'importance': tp.importance.RandomImportance()
            },
        },
        'pruning_ratios': [0.0, 0.2, 0.5, 0.7],
        'hidden_sizes': [512, 512, 512,128, 64, 32],      # Small proven architecture from literature
        'dropout_rate': 0.1,           # Lower dropout - less regularization needed
        'window_size': 30,             # Proven optimal window size
        'batch_size': 16,              # Smaller batches for better convergence
        'learning_rate': 0.005,        # Optimized learning rate
        'epochs': 1000,                # More epochs for convergence
        'patience': 30,                # More patience for convergence
        'output_dir': './results_mlp_nasa_simple',
        'models_dir': './models_mlp_nasa_simple',
        'data_dir': './data/CMaps'
    }

    os.makedirs(config['output_dir'], exist_ok=True)
    os.makedirs(config['models_dir'], exist_ok=True)

    # Simple data loading
    train_loader, val_loader, test_loader, input_size = get_data_loaders_simple(
        data_dir=config['data_dir'],
        batch_size=config['batch_size'],
        window_size=config['window_size']
    )

    example_input_cpu = torch.randn(1, input_size)
    example_input_device = example_input_cpu.to(DEVICE)
    criterion = nn.MSELoss()

    # Simple baseline training with optimized parameters
    print("\nTraining baseline with simple but optimized approach...")
    model = get_simple_mlp_model(input_size, config['hidden_sizes'], config['dropout_rate'])
    model.to(DEVICE)

    # Optimized simple optimizer - research shows this works best for NASA dataset
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=1e-6)

    trained_model, _ = train_model_simple(
        model, train_loader, criterion, optimizer, DEVICE,
        config['epochs'], val_loader, config['patience'], "Optimized Simple Baseline"
    )

    # Save and evaluate baseline
    baseline_path = os.path.join(config['models_dir'], 'baseline_model.pth')
    save_model(trained_model, baseline_path, example_input_cpu)

    baseline_metrics = evaluate_model(trained_model, test_loader, example_input_device, criterion, DEVICE)
    print(f"\nSimple Baseline Results:")
    print(f"  MSE: {baseline_metrics['mse']:.2f}")
    print(f"  MAE: {baseline_metrics['mae']:.2f}")
    print(f"  MACs: {baseline_metrics['macs']/1e6:.2f}M")
    print(f"  Params: {baseline_metrics['params']/1e6:.2f}M")

    # Initialize results
    all_results = {}
    for strategy_name in config['strategies'].keys():
        all_results[strategy_name] = {0.0: baseline_metrics}

    ignored_layers = get_ignored_layers(trained_model)

    # Simple pruning experiments
    print("\nStarting simple pruning experiments...")
    for strategy_name, strategy_config in config['strategies'].items():
        print(f"\n--- Strategy: {strategy_name} ---")

        for sparsity_ratio in config['pruning_ratios']:
            if sparsity_ratio == 0.0:
                continue

            print(f"\nTesting {strategy_name} at {sparsity_ratio:.1%} sparsity...")

            # Load fresh copy
            model_copy = get_simple_mlp_model(input_size, config['hidden_sizes'], config['dropout_rate'])
            model_copy.load_state_dict(torch.load(baseline_path, map_location=DEVICE))
            model_copy.to(DEVICE)

            # Simple pruning
            pruned_model = prune_model(model_copy, strategy_config, sparsity_ratio,
                                     example_input_device, ignored_layers)

            # Optimized fine-tuning
            print("Optimized fine-tuning...")
            optimizer_ft = optim.Adam(pruned_model.parameters(), lr=config['learning_rate']/2, weight_decay=1e-6)

            fine_tuned_model, _ = train_model_simple(
                pruned_model, train_loader, criterion, optimizer_ft, DEVICE,
                config['epochs']//2, val_loader, config['patience'],
                f"{strategy_name}-{sparsity_ratio:.1%}"
            )

            # Evaluate
            final_metrics = evaluate_model(fine_tuned_model, test_loader, example_input_device, criterion, DEVICE)
            all_results[strategy_name][sparsity_ratio] = final_metrics

            print(f"Results: MSE={final_metrics['mse']:.2f}, MAE={final_metrics['mae']:.2f}")

            # Save
            model_filename = f"{strategy_name.lower()}_sparsity_{sparsity_ratio:.1f}.pth"
            save_model(fine_tuned_model, os.path.join(config['models_dir'], model_filename), example_input_cpu)

    # Results
    summary_df = save_results_to_files(all_results, config['output_dir'])
    create_results_plots(summary_df, config['output_dir'])
    print_results_table(summary_df)

    # Analysis
    best_mse = summary_df['mse'].min()
    print(f"\nBest MSE achieved: {best_mse:.2f}")

    print(f"\n📁 Results: {os.path.abspath(config['output_dir'])}")
    print(f"📁 Models: {os.path.abspath(config['models_dir'])}")

if __name__ == "__main__":
    main()


Using device: cuda
Starting SIMPLE NASA MLP Approach - Back to Proven Basics
Loading NASA C-MAPSS dataset from: ./data/CMaps
--- Simple Proven Data Preparation ---
Removing 10 non-informative features
Keeping sensors: [2,3,4,7,8,9,11,12,13,14,15,17,20,21]
Using 14 proven effective sensors
Created 17731 samples with simple windowing
Created 100 samples with simple windowing
Input size: 420 (window: 30 × features: 14)
Train: 14185, Val: 3546, Test: 100

Training baseline with simple but optimized approach...
✅ Created Simple MLP: 420 -> 512 -> 512 -> 512 -> 128 -> 64 -> 32 -> 1
Epoch 1: Train: 949.4472, Val: 1923.2146
Early stopping at epoch 46
Loaded best model state
✅ Model saved to ./models_mlp_nasa_simple/baseline_model.pth

Simple Baseline Results:
  MSE: 211.03
  MAE: 10.55
  MACs: 0.82M
  Params: 0.82M

Starting simple pruning experiments...

--- Strategy: MagnitudeL2 ---

Testing MagnitudeL2 at 20.0% sparsity...
✅ Created Simple MLP: 420 -> 512 -> 512 -> 512 -> 128 -> 64 -> 32 ->